In [1]:
import pandas as pd
import seaborn as sns
from aequitas.audit import Audit
from aequitas.fairness import Fairness
import aequitas.plot as ap

# import warnings; warnings.simplefilter('ignore')

%matplotlib inline

c:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Exception: Cannot find lightgbm library file in following paths:
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\site-packages\fairgbm\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\site-packages\fairgbm\compile\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\site-packages\compile\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\lib\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\site-packages\compile\Release\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\site-packages\compile\windows\x64\DLL\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\Release\lib_lightgbm.dll
C:\Users\danie\Desktop\MIA\IAS\project\compas-machine-bias\.conda\Lib\windows\x64\DLL

In [ ]:
df = pd.read_csv("https://github.com/dssg/aequitas/raw/master/examples/data/compas_for_aequitas.csv")
df.head()